In [ ]:
pip install ipywidgets

In [ ]:
pip install huggingface_hub

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
from langchain_community.llms import OpenLLM

llm = OpenLLM(
    model_name="dolly-v2",
    model_id="databricks/dolly-v2-3b",
    temperature=0.94,
    repetition_penalty=1.2,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

instruct_pipeline.py:   0%|          | 0.00/9.16k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

NOT RECOMMENDED in production and SHOULD ONLY used for development.
c:\Users\Saqla\.conda\envs\ol\Lib\site-packages\transformers\generation\configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.94` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
%pip install --upgrade --quiet  openllm

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install vllm

In [5]:
import openllm


llm = openllm.LLM('databricks/dolly-v2-3b', embedded=True)

It is recommended to specify the backend explicitly. Cascading backend might lead to unexpected behaviour.
NOT RECOMMENDED in production and SHOULD ONLY used for development.
c:\Users\Saqla\.conda\envs\ol\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
import asyncio

import nest_asyncio


nest_asyncio.apply()


async def main():
  async for gen in llm.generate_iterator('What is the weather like in kashmir?', max_new_tokens=128, top_k=1):
    print(gen.outputs[0].text, flush=True, end='')


asyncio.run(main())



Kashmir has a very pleasant weather. The summers are very hot and the winters are cool. The winters are very popular among the tourists for skiing.

What is the best time to visit Kashmir?

The best time to visit Kashmir is between June and September. The weather is very pleasant and the winters are very popular for skiing.

What is the best time to visit Kashmir?

The best time to visit Kashmir is between June and

In [ ]:
!nohup openllm start databricks/dolly-v2-3b --port 8001 --dtype float16 --backend vllm > openllm.log 2>&1 &

In [ ]:
! curl -i http://127.0.0.1:8001/readyz

In [ ]:
!openllm query --endpoint http://127.0.0.1:8001 --timeout 120 "What is the weight of the earth?"

In [ ]:
#Open llms builtin python client

import openllm


# sync API
client = openllm.HTTPClient('http://127.0.0.1:8001', timeout=120)
res = client.generate('What is the weight of the earth?', max_new_tokens=8192)

# Async API
# async_client = openllm.AsyncHTTPClient("http://127.0.0.1:8001", timeout=120)
# res = await async_client.generate("what is the weight of the earth?", max_new_tokens=8192)
print(res.outputs[0].text)

In [ ]:
# streaming
for it in client.generate_stream('What is the weight of the earth?', max_new_tokens=2048, n=2, best_of=2):
  #print(f'index {it.index}: {it.text} (token: {it.token_ids})')
  print(it.text, end="")

In [ ]:
#open ai compatible endpoint

import openai
import os


client = openai.OpenAI(base_url=os.getenv('OPENLLM_ENDPOINT', 'http://localhost:8001') + '/v1', api_key='na')
models = client.models.list()
print('Models:', models.model_dump_json(indent=2))
model = models.data[0].id

#os.environ["STREAM"] = "TRUE"
stream = str(os.getenv('STREAM', False)).upper() in ['TRUE', '1', 'YES', 'Y', 'ON']

completions = client.completions.create(
  prompt='Write me a tag line for an ice cream shop.', model=model, max_tokens=64, stream=stream
)

print(f'Completion result (stream={stream}):')
if stream:
  for chunk in completions:
    text = chunk.choices[0].text
    if text:
      print(text, flush=True, end='')
else:
  print(completions.choices[0].text)

In [ ]:
from langchain.llms import OpenLLM


llm = OpenLLM(server_url='http://localhost:8001')

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


template = 'What is a good name for a company that makes {product}?'

prompt = PromptTemplate(template=template, input_variables=['product'])

llm_chain = LLMChain(prompt=prompt, llm=llm)

generated = llm_chain.run(product='mechanical keyboard')
print(generated)